In [ ]:
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, SkyOffsetFrame
from regions import CircleSkyRegion
import numpy as np
import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

# #import nustar_dem_prep as nu
import images_and_coalignment as iac
import time_interval_selection as tis
import nustar_utilities as nuutil
import gauss2D as g2d
import nustar_dem_prep as nu


# import os
# # For macOS
# def play_sound():
#     os.system("afplay /System/Library/Sounds/Glass.aiff")

import pickle
import pathlib
import importlib
from astropy import units as u
import numpy as np


with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

keys = list(data.keys())

In [ ]:
aia_dir = '/Users/jmdunca2/sample_aia/sample_aia/'

all_maps=[]
for k in keys:
    print(k)
    if k == '22-apr-16_1':
        continue

    ARDict = data[k]

    id_dirs = ARDict['datapaths']
    aiamaps = iac.get_orbit_aiamaps(aia_dir, id_dirs, wave=94)   
    all_maps.append(aiamaps[0])
    
    # obsids = ARDict['obsids']
    # working_dir = ARDict['working_dir']

    # if method=='double':
    #     gauss_stats = ARDict['gauss_stats']
    #     sep_axis = gauss_stats[0][0]
    # else:
    #     sep_axis = ''

In [ ]:
len(all_maps)

In [ ]:
all_regiondicts = []

for k in keys:
    
    regiondicts=[]
    
    if k == '22-apr-16_1':
        continue

    ARDict = data[k]
    method = ARDict['method']
    print(k)
    orbdirs = glob.glob(ARDict['prepped_aia']+'*')
    orbdirs.sort()
    #print(orbdirs)
    
    timefiles = glob.glob(orbdirs[0]+'/*')
    timefiles.sort()
    print(timefiles[0])
    
    with open(timefiles[0], 'rb') as f:
        aiadata = pickle.load(f)

    if 'region0' in aiadata.keys():
        try:
            print('Reg 0: ', aiadata['region0'].keys())
        except AttributeError:
            #for i in range(1,14):
            with open(timefiles[12], 'rb') as f:
                aiadata = pickle.load(f)
            print('Reg 0: ', aiadata['region0'].keys())

        regiondicts.append(aiadata['region0'])
            #print('Reg 0: ', aiadata['region0'].keys())
            

    if 'region1' in aiadata.keys():
        print('Reg 1: ', aiadata['region1'].keys())
        regiondicts.append(aiadata['region1'])

    if 'radius' in aiadata.keys():
        print(aiadata.keys())
        regiondicts.append(aiadata)


    all_regiondicts.append(regiondicts)


    print('')

In [ ]:
len(all_regiondicts)

In [ ]:
(30/4)*7

In [ ]:
noindices = [0, 5, 9, 14, 15, 19, 20, 21]

fig = plt.figure(figsize=(40,22))

howmany=1
for i in range(0, len(all_regiondicts)):

    if i not in noindices:
        #print(i)

        m = all_maps[i]
    
        regd = all_regiondicts[i]
        if len(regd) == 1:
            xx = regd[0]['centerx'].value
            yy = regd[0]['centery'].value
        else:
            xx = (regd[0]['centerx'].value + regd[1]['centerx'].value)/2.
            yy = (regd[0]['centery'].value + regd[1]['centery'].value)/2.
            
    
        #Set broad box for plotting (using region object)
        bl=[(xx-275)*u.arcsec, (yy-275)*u.arcsec]
        tr=[(xx+275)*u.arcsec,(yy+275)*u.arcsec]
        #print(tr[0]-bl[0], tr[1]-bl[1])
    
        bottom_left = SkyCoord(bl[0]-100*u.arcsec, bl[1]-100*u.arcsec, frame=m.coordinate_frame)
        top_right = SkyCoord(tr[0]+100*u.arcsec,tr[1]+100*u.arcsec, frame=m.coordinate_frame)
        mm = m.submap(bottom_left=bottom_left, top_right=top_right)
    
        
        ax = fig.add_subplot(4,7,howmany, projection=mm)
        ax.set_axis_off()

        norm = mm.plot_settings['norm']
        #print(norm.vmin, norm.vmax)
        norm.vmin, norm.vmax = 0.75, 60 #np.percentile(mm.data, [1, 99.9])
        #print(norm.vmin, norm.vmax)
        
        mm.plot(axes=ax, norm=norm)
        ax.set_title(mm.name[0:6]+r'$\AA$'+mm.name[17:-3], fontsize=20)
        howmany+=1

        for r in regd:
            region = iac.make_region(r, mm)
            og_region = region.to_pixel(mm.wcs)                    
            og_region.plot(axes=ax, color='red', ls='--', lw=3)

        #print('')
            
        

plt.savefig('All_regions_AIA94_reference.png')

In [ ]:
dir(mm)

In [ ]:
mm.name[0:9]+r'$\AA$'+mm.name[17:]

In [ ]:
print(r'Wavelength (\AA)')

In [ ]:
for i in range(0, len(keys)):
    print(i, keys[i])